## Downloading the jp2 file section

In [ ]:
import gdown
import os

url  = "https://drive.google.com/file/d/1Hb7Lgo5vtbAtr36fq6knylSiATlJRDS6/view?usp=share_link"
output = "./storage/jp2cache/B_37_FB3-SL_570-ST_NISL-SE_1708_lossless.jp2"


model_url = "https://drive.google.com/file/d/1SbSArI3KOOWHxRlxnjchO7_MbWzB4lNR/view"
model_output = "hovernet_fast_pannuke_type_tf2pytorch.tar"


storage_dir = "./storage"

if not os.path.exists(storage_dir + "/jp2cache"):
    os.makedirs(storage_dir + "/jp2cache")

In [ ]:
gdown.download(url=url, output=output, quiet=False, fuzzy=True)

## Model checkpoint

In [ ]:
gdown.download(url=model_url, output=model_output, quiet=False, fuzzy=True)

In [ ]:
!ls

## Importing the neccesary libs to access the jp2 file

In [ ]:
from jp2tileaccesor.multi_res_Tiling import SectionProxy, TileAccessor, Span

In [ ]:
proxy_lossless = SectionProxy(15, "NISSL", 1708)

print(proxy_lossless)

In [ ]:
proxy_lossless.check_local_jp2()

In [ ]:
Image_size = 256
size = Span(Image_size,Image_size)
accessor = TileAccessor(proxy_lossless, 0, tilespan = size, use_iip = False)
print(accessor)

In [ ]:
tile = accessor[np.random.randint(accessor.ntiles)]
plt.imshow(tile[0])

In [ ]:
accessor[np.random.randint(accessor.ntiles)][0].shape

* Randomly sampling the images to make a dataset to test all models on.
* Tiles are selected at random because continous selection can lead to rows of tiles which do not contain brain matter.

In [ ]:
num_tiles = 1024
t=0
check_make_dir(storage_dir + "/cache_" + str(Image_size))

for i in range(0,accessor.ntiles):
    if t >= num_tiles:
        break
    tile = accessor[np.random.randint(accessor.ntiles)]
    if tile[0].shape == (256, 256, 3):
        plt.imsave(storage_dir+"/cache_"+str(Image_size) + "/"+str(i)+".png",tile[0])
        t+=1

In [ ]:
file_list = os.listdir(storage_dir+"/cache_"+str(Image_size))
print("Number of Images",len(file_list))
pics = [ storage_dir + "/cache_"+str(Image_size) + "/" +str(i) for i in file_list]

In [ ]:
import numpy as np
np_pics = np.asarray(Image.open(pics[0]))
brain_tiles = np_pics.reshape(1,4,Image_size,Image_size)
for i in range(len(pics)-1):
    img = np.asarray(Image.open(pics[i+1]))
    if (img.shape == (256, 256, 4)):
        brain_tiles = np.concatenate((brain_tiles,img.reshape(1,4,Image_size,Image_size)),axis=0)
    

In [ ]:
brain_tiles = torch.from_numpy(brain_tiles).float()

In [ ]:
brain_tiles = brain_tiles[:1024,:3,:,:]

In [ ]:
brain_tiles.shape

## Importing the original Hovernet model

In [1]:
import os
from PIL import Image
import numpy as np
import tensorrt
import onnx
import gc
import models.hovernet.net_desc as net
import torch
import matplotlib.pyplot as plt
torch.set_grad_enabled(False)
Image_size = 256
def check_make_dir(path_dir):
    if not os.path.exists(path_dir):
        os.makedirs(path_dir)


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from run_utils.utils import convert_pytorch_checkpoint
model_path = 'hovernet_fast_pannuke_type_tf2pytorch.tar'
hovernet = net.HoVerNet(nr_types = 6,mode='fast')
saved_state_dict = torch.load(model_path)["desc"]
saved_state_dict = convert_pytorch_checkpoint(saved_state_dict)

hovernet.load_state_dict(saved_state_dict, strict=True)
hovernet.eval()

HoVerNet(
  (conv0): Sequential(
    (pad): TFSamepaddingLayer()
    (/): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
  )
  (d0): ResidualBlock(
    (units): ModuleList(
      (0): Sequential(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (conv1/bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1/relu): ReLU(inplace=True)
        (conv2/pad): TFSamepaddingLayer()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (conv2/bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2/relu): ReLU(inplace=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (1): Sequential(
        (preact/bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, 

In [8]:
import torch_tensorrt

In [9]:
inputs = [
    torch_tensorrt.Input(
        min_shape=[16, 3, 256, 256],
        opt_shape=[16, 3, 256, 256],
        max_shape=[16, 3, 256, 256],
        dtype=torch.float32,
    )
]
enabled_precisions = {torch.float, torch.half}  # Run with fp16


In [ ]:
trt_ts_module = torch_tensorrt.ts.compile(
    traced, inputs=inputs, enabled_precisions=enabled_precisions,
    truncate_long_and_double = True
)

In [ ]:
type(trt_ts_module)

In [6]:
f = open('trt_ts_256_16.ts', 'wb')

In [12]:
f.write(torch_tensorrt.ts.convert_method_to_trt_engine(
    traced, inputs=inputs, enabled_precisions=enabled_precisions,
    truncate_long_and_double = True, method_name = "forward"
))
f.close()

ERROR: [Torch-TensorRT] - Method requested cannot be compiled end to end by Torch-TensorRT.TorchScript.
Unsupported operators listed below:
  - aten::Int.Tensor(Tensor a) -> int
  - aten::tensordot(Tensor self, Tensor other, int[] dims_self, int[] dims_other) -> Tensor
  - aten::pad(Tensor self, int[] pad, str mode="constant", float? value=None) -> Tensor
You can either implement converters for these ops in your application or request implementation
https://www.github.com/nvidia/Torch-TensorRT/issues

In Module:

ERROR: [Torch-TensorRT] - Unsupported operator: aten::Int.Tensor(Tensor a) -> int

ERROR: [Torch-TensorRT] - Unsupported operator: aten::tensordot(Tensor self, Tensor other, int[] dims_self, int[] dims_other) -> Tensor
/usr/local/lib/python3.8/dist-packages/torch/functional.py(1092): tensordot
/workspace/trt_hovernet/models/hovernet/net_utils.py(306): forward
/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py(1178): _slow_forward
/usr/local/lib/python3.8/dist-pa

RuntimeError: [Error thrown at core/compiler.cpp:267] Expected conversion::VerifyConverterSupportForBlock(g->block()) to be true but got false
Not all operations in graph are supported by the compiler


al/lib/python3.8/dist-packages/torch/nn/modules/container.py(204): forward
/workspace/trt_hovernet/models/hovernet/net_utils.py(277): forward
/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py(1178): _slow_forward
/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py(1190): _call_impl
/workspace/trt_hovernet/models/hovernet/net_desc.py(116): forward
/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py(1178): _slow_forward
/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py(1190): _call_impl
/usr/local/lib/python3.8/dist-packages/torch/jit/_trace.py(976): trace_module
/tmp/ipykernel_259093/4018293670.py(6): <module>
/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py(3433): run_code
/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py(3373): run_ast_nodes
/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py(3194): run_cell_async
/usr/local/lib/python3.8/dist-packages/IPython/c

In [4]:
batch=16
size = 256
dummy_input = torch.rand(batch,3,size,size)
hovernet = hovernet.to("cuda")
dummy_input = dummy_input.to('cuda')
traced = torch.jit.trace_module(hovernet,{"forward":dummy_input})

In [5]:
type(traced)

torch.jit._trace.TopLevelTracedModule